We need to modify the LAMMPS input files such that:
1. does **not** change the unit cell parameters during minimization by
    removing the lines:
```
fix        1 all box/relax aniso 0.0 vmax 0.01
minimize   1.0e-15 1.0e-15 10000 100000
unfix      1
min_style  fire
``` 

2. write the unit cell parameters vol, lx, ly, lz, xy, xz, yz to a .csv by editing 
    the print statements to the be:
```
print "MinStep,CellMinStep,AtomMinStep,FinalStep,Energy,EDiff,vol,lx,ly,lz,xy,xz,yz" file xtalname.min.csv screen no
```
and
```
print "${iter}, ${CellMinStep}, ${AtomMinStep}, ${AtomMinStep}, $(pe), ${min_E}, $(vol), $(lx), $(ly), $(lz), $(xy), $(xz), $(yz)" append xtalname.min.csv screen no 
```

In [1]:
# unit_cell_rep = "1x1x1"
# cutoff_radius = 5.0
# dump_xyz = 20000
# run(`lammps-interface --minimize --cutoff $cutoff_radius --replication $unit_cell_rep --xyz $dump_xyz --restart $xtal`)

In [2]:
using PorousMaterials
using Printf

In [3]:
function find_edit_indexes(lines::Array{String,1})
    # find the indexes of the lines we need to remove or edit
    n = []
    for (i, line) in enumerate(lines)
        if occursin("box/relax", line)
            push!(n, i - 1)
            push!(n, i)
            push!(n, i + 1)
            push!(n, i + 2)
        elseif occursin("print", line)
            push!(n, i)
        elseif occursin("min_style", line)
            push!(n, i)
        end
    end
    return n
end


function mod_print_line(line::String)
    if occursin("\$", line)
        a = split(line, '\"')
        a[end - 1] *= ",\$(vol),\$(lx),\$(ly),\$(lz),\$(xy),\$(xz),\$(yz)"
        newline = a[1] * "\"" * a[2] * "\"" * a[3]
        return newline
    else
        a = split(line, '\"')
        a[end - 1] *= ",vol,lx,ly,lz,xy,xz,yz"
        newline = a[1] * "\"" * a[2] * "\"" * a[3]
        return newline
    end
end

mod_print_line (generic function with 1 method)

In [4]:
# # The name of the fragment
# # file must be a .xyz file located in ./fragments/
# # The atom species that is bonded to the
# # :C_aro_R atom on the ring segment of the Fragment
fragments_with_overlap = ["C-3CH3", "CH2-CH2-CH3", "CH2-CH3","CH2-NH2",
                          "CH-CH2", "N-2CH3", "O-CH2-CH2-CH3",
                          "O-CH2-CH3", "O-CH3", "P-2CH3", "PH2"]

# fragments_with_overlap = ["CH2-NH2"]

# # The type of Arene Substitution ("ortho", "meta", "all") 
# # TODO: "all" not yet implimented
substitution_types = ["meta", "ortho"]

2-element Array{String,1}:
 "meta"
 "ortho"

In [6]:
###
#  Flag for running lammps_interface with set args
###
run_lammps_interface = true
unit_cell_rep = "1x1x1"
cutoff_radius = 5.0
dump_xyz = 20000

###
#  Flag for running minimization after generating input.xtalname
###
run_minimization = true

###
#  Loop over structures and generate input files
###
for fragment in fragments_with_overlap
    # move into dir containing lammps sims of fragment
    cd(fragment)
    println(pwd())
    for sub_type in substitution_types
        structurename = "NiPyC2_relax_sc211_" * sub_type * "_functionalized_" * fragment
        
        if run_lammps_interface
            xtal = joinpath("../../mof_construction/NiPyC2_relax_sc211/", structurename * ".cif")
            
            run(`lammps-interface --minimize --cutoff $cutoff_radius --replication $unit_cell_rep --xyz $dump_xyz --restart $xtal`)
        end
        
        filename = "in." * structurename
        lines = readlines(filename)
        
        # find indecies of specific lines in file
        inds  =  find_edit_indexes(lines)
        
        # name and open file to write 
        input_filename = joinpath("input." * structurename)
        input_file = open(input_filename, "w")
        
        # loop over lines and write to file
        for (mm, line) in enumerate(lines)
            if mm in inds
                if occursin("print", line)
                    newline = mod_print_line(line)
                    @printf(input_file,
                        "%s\n"
                        , newline)
                elseif occursin("min_style", line)
                    # if the line is "min_style fire" skip it
                    # otherwise write "min_style cg"
                    if occursin("fire", line)
                        continue
                    else 
                        @printf(input_file,
                            "%s\n", 
                            "min_style       cg")
                    end
                else
                    continue
                end
            else
                @printf(input_file, "%s\n", line)
            end
        end
        close(input_file)
        
        # run LAMMPS minimization using inpout file
        if run_minimization
            run(`lmp_stable -in $input_filename`)
        end
    end
    # move back into parent dir
    cd("..")
    @assert pwd() == "/home/ng/DTRA/lammps_sims"
end

/home/ng/DTRA/lammps_sims/C-3CH3
totatomlen = 204
compute_topology_information()
func: cartesian_coordinates; Elps. 0.003s
func: min_img_distances; Elps. 0.524s
func: compute_bonding; Elps. 0.557s
func: init_typing; Elps. 0.596s
func: bond_typing; Elps. 0.598s
func: angles; Elps. 0.599s
func: dihedrals; Elps. 0.600s
func: improper_dihedrals; Elps. 0.601s
Molecules found in the framework, separating.
Files created! -> /home/ng/DTRA/lammps_sims/C-3CH3
LAMMPS (3 Mar 2020)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:94)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0 0 0) to (12.5056 12.5234 10.2743) with tilt (0 -0.227594 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  204 atoms
  scanning bonds ...
  6 = max bonds/atom
  scanning angles ...
  15 = max angles/atom
  scanning dihedrals ...
  27 = max dihedrals/atom
  scanning impropers ...
  3 = max impropers/atom
  reading bonds ...
  207 bonds
  reading angles

totatomlen = 204
compute_topology_information()
func: cartesian_coordinates; Elps. 0.004s
func: min_img_distances; Elps. 0.536s
func: compute_bonding; Elps. 0.568s
func: init_typing; Elps. 0.608s
func: bond_typing; Elps. 0.611s
func: angles; Elps. 0.612s
func: dihedrals; Elps. 0.613s
func: improper_dihedrals; Elps. 0.614s
Molecules found in the framework, separating.
Files created! -> /home/ng/DTRA/lammps_sims/C-3CH3
LAMMPS (3 Mar 2020)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:94)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0 0 0) to (12.5056 12.5234 10.2743) with tilt (0 -0.227594 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  204 atoms
  scanning bonds ...
  6 = max bonds/atom
  scanning angles ...
  15 = max angles/atom
  scanning dihedrals ...
  27 = max dihedrals/atom
  scanning impropers ...
  3 = max impropers/atom
  reading bonds ...
  207 bonds
  reading angles ...
  401 angles
  reading dihed

totatomlen = 180
compute_topology_information()
func: cartesian_coordinates; Elps. 0.003s
func: min_img_distances; Elps. 0.406s
func: compute_bonding; Elps. 0.436s
func: init_typing; Elps. 0.472s
func: bond_typing; Elps. 0.474s
func: angles; Elps. 0.475s
func: dihedrals; Elps. 0.476s
func: improper_dihedrals; Elps. 0.477s
Files created! -> /home/ng/DTRA/lammps_sims/CH2-CH2-CH3
LAMMPS (3 Mar 2020)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:94)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0 0 0) to (12.5056 12.5234 10.2743) with tilt (0 -0.227594 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  180 atoms
  scanning bonds ...
  6 = max bonds/atom
  scanning angles ...
  15 = max angles/atom
  scanning dihedrals ...
  14 = max dihedrals/atom
  scanning impropers ...
  3 = max impropers/atom
  reading bonds ...
  200 bonds
  reading angles ...
  388 angles
  reading dihedrals ...
  448 dihedrals
  reading improp

Setting up Verlet run ...
  Unit style    : real
  Current step  : 0
  Time step     : 1
Per MPI rank memory allocation (min/avg/max) = 17.68 | 17.68 | 17.68 Mbytes
Step Temp E_pair E_mol TotEng Press 
       0            0    270.08248    2024.6131    2294.6956   -722.64602 
Loop time of 7.42992e-07 on 1 procs for 0 steps with 180 atoms

269.2% CPU use with 1 MPI tasks x 1 OpenMP threads

MPI task timing breakdown:
Section |  min time  |  avg time  |  max time  |%varavg| %total
---------------------------------------------------------------
Pair    | 0          | 0          | 0          |   0.0 |  0.00
Bond    | 0          | 0          | 0          |   0.0 |  0.00
Neigh   | 0          | 0          | 0          |   0.0 |  0.00
Comm    | 0          | 0          | 0          |   0.0 |  0.00
Output  | 0          | 0          | 0          |   0.0 |  0.00
Modify  | 0          | 0          | 0          |   0.0 |  0.00
Other   |            | 7.43e-07   |            |       |100.00

Nlocal:   

Step Temp E_pair E_mol TotEng Press 
     406            0    301.14904    2292.9975    2594.1466   -15901.067 
     407            0    301.14904    2292.9975    2594.1466   -15901.067 
Loop time of 0.0149479 on 1 procs for 1 steps with 180 atoms

100.0% CPU use with 1 MPI tasks x 1 OpenMP threads

Minimization stats:
  Stopping criterion = linesearch alpha is zero
  Energy initial, next-to-last, final = 
         2594.14657151      2594.14657151      2594.14657151
  Force two-norm initial, final = 7.02528 7.02528
  Force max component initial, final = 1.56828 1.56828
  Final line search alpha, max atom move = 2.3754e-10 3.72529e-10
  Iterations, force evaluations = 1 56

MPI task timing breakdown:
Section |  min time  |  avg time  |  max time  |%varavg| %total
---------------------------------------------------------------
Pair    | 0.0097716  | 0.0097716  | 0.0097716  |   0.0 | 65.37
Bond    | 0.0043443  | 0.0043443  | 0.0043443  |   0.0 | 29.06
Neigh   | 0          | 0          | 0

...

Setting up Verlet run ...
  Unit style    : real
  Current step  : 0
  Time step     : 1
Per MPI rank memory allocation (min/avg/max) = 16.4 | 16.4 | 16.4 Mbytes
Step Temp E_pair E_mol TotEng Press 
       0            0    36.428713    1725.5017    1761.9304   -81792.595 
Loop time of 7.90009e-07 on 1 procs for 0 steps with 140 atoms

253.2% CPU use with 1 MPI tasks x 1 OpenMP threads

MPI task timing breakdown:
Section |  min time  |  avg time  |  max time  |%varavg| %total
---------------------------------------------------------------
Pair    | 0          | 0          | 0          |   0.0 |  0.00
Bond    | 0          | 0          | 0          |   0.0 |  0.00
Neigh   | 0          | 0          | 0          |   0.0 |  0.00
Comm    | 0          | 0          | 0          |   0.0 |  0.00
Output  | 0          | 0          | 0          |   0.0 |  0.00
Modify  | 0          | 0          | 0          |   0.0 |  0.00
Other   |            | 7.9e-07    |            |       |100.00

Nlocal:    14

totatomlen = 172
compute_topology_information()
func: cartesian_coordinates; Elps. 0.003s
func: min_img_distances; Elps. 0.383s
func: compute_bonding; Elps. 0.414s
func: init_typing; Elps. 0.451s
func: bond_typing; Elps. 0.453s
func: angles; Elps. 0.454s
func: dihedrals; Elps. 0.455s
func: improper_dihedrals; Elps. 0.456s
Files created! -> /home/ng/DTRA/lammps_sims/P-2CH3
LAMMPS (3 Mar 2020)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:94)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0 0 0) to (12.5056 12.5234 10.2743) with tilt (0 -0.227594 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  172 atoms
  scanning bonds ...
  6 = max bonds/atom
  scanning angles ...
  15 = max angles/atom
  scanning dihedrals ...
  12 = max dihedrals/atom
  scanning impropers ...
  3 = max impropers/atom
  reading bonds ...
  192 bonds
  reading angles ...
  364 angles
  reading dihedrals ...
  384 dihedrals
  reading impropers .

Per MPI rank memory allocation (min/avg/max) = 18.17 | 18.17 | 18.17 Mbytes
Step Temp E_pair E_mol TotEng Press 
     204            0    95.236134    1957.0991    2052.3352   -41728.233 
     205            0    95.236134    1957.0991    2052.3352   -41728.233 
Loop time of 0.0133918 on 1 procs for 1 steps with 172 atoms

98.3% CPU use with 1 MPI tasks x 1 OpenMP threads

Minimization stats:
  Stopping criterion = linesearch alpha is zero
  Energy initial, next-to-last, final = 
         2052.33524378      2052.33524378      2052.33524378
  Force two-norm initial, final = 5.42861 5.42861
  Force max component initial, final = 1.16958 1.16958
  Final line search alpha, max atom move = 3.18515e-10 3.72529e-10
  Iterations, force evaluations = 1 56

MPI task timing breakdown:
Section |  min time  |  avg time  |  max time  |%varavg| %total
---------------------------------------------------------------
Pair    | 0.0085886  | 0.0085886  | 0.0085886  |   0.0 | 64.13
Bond    | 0.0040267  | 0

totatomlen = 124
compute_topology_information()
func: cartesian_coordinates; Elps. 0.023s
func: min_img_distances; Elps. 0.305s
func: compute_bonding; Elps. 0.328s
func: init_typing; Elps. 0.360s
func: bond_typing; Elps. 0.362s
func: angles; Elps. 0.362s
func: dihedrals; Elps. 0.364s
func: improper_dihedrals; Elps. 0.364s
Files created! -> /home/ng/DTRA/lammps_sims/PH2
LAMMPS (3 Mar 2020)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:94)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0 0 0) to (12.5056 12.5234 10.2743) with tilt (0 -0.227594 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  124 atoms
  scanning bonds ...
  6 = max bonds/atom
  scanning angles ...
  15 = max angles/atom
  scanning dihedrals ...
  12 = max dihedrals/atom
  scanning impropers ...
  3 = max impropers/atom
  reading bonds ...
  144 bonds
  reading angles ...
  268 angles
  reading dihedrals ...
  288 dihedrals
  reading impropers ...


totatomlen = 124
compute_topology_information()
func: cartesian_coordinates; Elps. 0.030s
func: min_img_distances; Elps. 0.335s
func: compute_bonding; Elps. 0.360s
func: init_typing; Elps. 0.391s
func: bond_typing; Elps. 0.393s
func: angles; Elps. 0.394s
func: dihedrals; Elps. 0.395s
func: improper_dihedrals; Elps. 0.395s
Files created! -> /home/ng/DTRA/lammps_sims/PH2
LAMMPS (3 Mar 2020)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:94)
  using 1 OpenMP thread(s) per MPI task
Reading data file ...
  triclinic box = (0 0 0) to (12.5056 12.5234 10.2743) with tilt (0 -0.227594 0)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  124 atoms
  scanning bonds ...
  6 = max bonds/atom
  scanning angles ...
  15 = max angles/atom
  scanning dihedrals ...
  8 = max dihedrals/atom
  scanning impropers ...
  3 = max impropers/atom
  reading bonds ...
  144 bonds
  reading angles ...
  268 angles
  reading dihedrals ...
  288 dihedrals
  reading impropers ...
 